# Vectores, operaciones vectoriales básicas y Universal functions



Como cientificos de datos en algunos casos requeriremos crear nuestras propias funciones y es recomendable que se apeguen a las ufuncs, es decir que se puedan aplicar independiente de la dimensión del objeto al que se aplican, que soporten broadcasting y muy importante **que su definición evite ciclos y use operaciones vectorizadas y otras ufuncs** .

In [1]:
import numpy as np

## Ejercicio 1
En data science y programación científica es común necesitar vectores con valores dentro de cierto intervalo, pero con sub-intervalos internos con diference espaciamiento ,por ejemplo:

[0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.7 , 0.8 , 0.9 , 1.  ]
       
El primer sub-intervalo incrementa de 0.1 en 0.1 , el segundo de 0.01 en 0.01 , y el tercero de 0.1 en 0.1 nuevamente.

Usando unicamente NumPy crea un vector de este tipo con el nombre xs .

**nota**: no es permitido usar ciclos, debe realizarse con operaciones "vectorizadas" de numpy

In [2]:
a=np.linspace(0,0.4,5,endpoint=True)
b=np.linspace(0.5,0.59,10,endpoint=True)
c=np.linspace(0.6,1,5,endpoint=True)
xs=np.hstack((a,b,c))
xs

array([0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.7 , 0.8 , 0.9 , 1.  ])

## Ejercicio 2
Suavizado de curvas a través de promedios de n-puntos(medias móviles):

<img src="https://www.luisllamas.es/wp-content/uploads/2017/03/arduino-filtro-media-movil-ventana-5.png">

En el mundo la data comunmente posee "ruido" por lo cual muchas veces antes de hacer análisis o modelado se necesita "suavizar" las curvas para reducir este ruido, una técnica muy sencilla es  el suavizado por promedio de n-puntos(también llamado media móvil), esto significa que para cada punto Xn obtenemos una versión transformada(suavizada) que consiste en promediar n puntos cercanos a el, por ejemplo para n= 3 tenemos que:

$$XS_{n}  =  \frac{X_{n-1}+X_{n}+X_{n+1}}{3}$$

In [3]:
# las siguientes 3 lineas generan un conjunto de datos que se comportan segun una onda senoidal pero tienen
# ruido por lo cual vamos a suavizar usando promedio de 3 puntos
# en este ejercicio los generamos manualmente pero pensemos que son datos que pudieron ser generados con algún
# instrumento como un sensor, o bien estar almacenados en una base de datos
ruido = 0.1*np.random.randn(15) #el ruido comunmente se debe a aleatoriedad o captura no exacta de info.
x = np.linspace(0,2*np.pi,15) + ruido
x = np.sin(x)
x
#xs es "x suavizado", inicialmente es una copia de x
xs =  x.copy()

for n in range(1,len(x)-1):
    xs[n] = (x[n-1] + x[n] + x[n+1])/3
print(x)
print(xs)


[-0.07791142  0.52302437  0.72094298  0.97715757  0.91854865  0.62350109
  0.50162318  0.0479558  -0.4834172  -0.78498051 -0.97449978 -0.97576898
 -0.742033   -0.29527361  0.13510498]
[-0.07791142  0.38868531  0.74037497  0.8722164   0.83973577  0.68122431
  0.39102669  0.02205393 -0.40681397 -0.7476325  -0.91174976 -0.89743392
 -0.6710252  -0.30073388  0.13510498]


En este caso usamos un ciclo  para ejemplificar pero ya hemos mencionado que debemos evitarlos siempre que sea posible y usar operaciones "vectorizadas" ya que es mucho mas rápido al utilizar  código pre-compilado de C, en este ejercicio debemos remplazar el calculo de xs para hacerlo de manera "vectorizada" y eliminar el ciclo for. 

**tip**: usar slicing

In [4]:
ruido = 0.1*np.random.randn(15) 
x = np.linspace(0,2*np.pi,15) + ruido
x = np.sin(x)

# xs es "x suavizado", inicialmente es una copia de x
xs =  x.copy()


xn=np.convolve(x, np.ones(3)/3, mode='valid');
xl=np.hstack((x[0],xn[:],x[-1]))

print(x)
print(xl)

[-0.00311131  0.435891    0.80083052  0.97256425  0.9063192   0.73285269
  0.45226155 -0.09316501 -0.37990334 -0.81387395 -0.92718889 -0.96098114
 -0.9025429  -0.53491945  0.07023614]
[-0.00311131  0.41120341  0.73642859  0.89323799  0.87057871  0.69714448
  0.36398308 -0.0069356  -0.42898077 -0.70698873 -0.90068133 -0.93023765
 -0.79948116 -0.45574207  0.07023614]


## Ejercicio 3

NumPy posee logaritmos naturales , base 2 y base 10 , crear una *ufunc* que calcule el logaritmo en cualquier base n dada, la funcion debe recibir el tensor al cual calcular el logaritmo elemento por elemento y la base del logaritmo a calcular, por ejemplo:

`def logaritmo_n(x,n):`

In [5]:
def logaritmo_n(x,n):
    return np.log(x)/np.log(n)


p=np.array([2,4,8,16])
logaritmo_n(p,2)




array([1., 2., 3., 4.])

## Ejercicio 4
Para calcular la información de una distribución de probabilidad necesitamos calcular el producto entre la información de cada posible $x$ y su probabilidad, cada uno de estos elementos nos indica cuanta incerteza aporta cada $x$ a la entropía de la distribución. 

Para completar este ejercicio hay que crear una función que calcule lo descrito anteriormente, debe recibir un vector representando a la dist. de probabilidad y devoler otro vector con el termino de entropía para cada $x$.

Por ejemplo:

`calcular_entropia([0.25,0.75])`

Debe resultar en :

$[0.5    ,    0.31127812]$

**Nota**: la entropía es la suma sobre estos valores, en este caso no calculamos la entropía,unicamente los termimos de su sumatoria. La entropía es:

$$S = -\sum_i^nP(x_{i})log_{2}P(x_{i})$$

In [6]:
## tu codigo aqui (~ 2 lineas de codigo)

def calcular_entropia(p):
    return p * np.log2(p) * (-1)
    
x=([0.25,0.75])
calcular_entropia(x)


array([0.5       , 0.31127812])

## Ejercicio 5

A partir de la información en un data warehouse se ha generado un modelo de datos, el valor de los clientes en el tiempo se ha determinado un modelo predictivo  no lineal que estima cuanta ganancia ha generado cierto cliente en 5 años en base a lo que ha gastado en su primer mes como cliente. 

El modelo tiene la forma:

$$g(x) = (x^{3} + 2x^{2} + e^{0.0001x} -\sqrt{2x})/50$$

Crear una función que calcule la ganancia para valores de gasto de nuevos clientes:

In [7]:
def g(x):
    ganancia = (np.power(x,3) + 2.0*(np.power(x,2.0))+np.exp(0.0001*x) - np.sqrt(2.0*x))/50
    
    return ganancia
    
gastos = np.array([100,50,25])
ganancia = g(gastos)

print("Ganancias",ganancia)

Ganancias [20399.73735829  2599.82010025   337.37862871]


Se ha determinado también que hay costos asociados a la comunicación y relación con los clientes(llamadas,mensajes electrónicos,etc) y que estos también tienen alta correlación con el monto que estos gastan en su primer més siguiendo el modelo:
$$c(x) = (x^{2} + x + log (0.0001x)-\sqrt{0.54x})/100$$

Cree una función para calcular este modelo sobre valores de de gasto de nuevos clientes , y luego cree una función  **ganancia_neta(x,g,c)** donde:
* x = vector conteniendo los valores de gasto del primer mes para nuevos clientes.
* g = función que estima la ganancia en función de x
* c = función que estima el costo en función de x

La función realiza el calculo simple **g(x) - c(x)** , ya que es posible que los modelos cambien en el tiempo es común recibir como parámetro el modelo o modelos hijos de un modelo mas grande , en este caso eso se logra recibiendo como parámetro las funciones de ganancia y costo.

In [8]:
def c(x):
    costo = (np.power(x,2)+x+np.log(0.0001*x)-np.sqrt(0.54*x)/100)
    return costo

gastos = np.array([100,50,25])
print(g(gastos))
print("Costos", c(gastos))


#Funcion de ganancia neta
def ganancia_neta(x,g,c):
    ganancia_neta = g(x)-c(x)
    return ganancia_neta

#ejecución de dicha funcion sobre el array "gastos"
print("Ganancia neta", ganancia_neta(gastos, g, c))




[20399.73735829  2599.82010025   337.37862871]
Costos [10095.32134512  2544.64972111   643.97179311]
Ganancia neta [10304.41601317    55.17037914  -306.5931644 ]


# Funciones miscelaneas(agregación y estadistica descriptiva básica) 

Ya hemos trabajado algunas funciones de NumPy pero han sido funciones "elementwise" cuyo vector resultante tiene el mismo tamaño que el vector de entrada, existen muchas otras funciones que utilizamos en casos en los que necesitamos agregar,resumir cierto vector,incluyendo en este tipo de funciones algunas de estadistica descriptiva básica.

## Ejercicio 6
Usando argmax crear una función `mode(x)` para calcular la moda de un vector $x$, esta función debe devolver una tupla de 2 elementos de la forma:

(valor,conteo)

**tip** investigar y auxiliarse de otras funciones de numpy y np.argmax, no olvidar trabajar todo vectorizado, sin ciclos.

In [9]:
p = ([1,2,3,4,3,2,3,2,1,3,2,3,4,1])

def mode(x):
    data = np.unique(x,return_index=True,return_counts=True)
    index = np.argmax(data[2])
    return np.array(data)[0,index],np.array(data)[2,index]
print(mode(p))

(3, 5)


## Ejercicio 7
Crear una función para calcular la entropía  de una distribución de probabilidad representada como un vector.
$$S = -\sum_i^nP(x_{i})log_{2}P(x_{i})$$

**Nota**: a diferencia del ejercicio anterior donde solo calculamos los términos de la entropía, en este caso  si calculamos el valor completo de la entropía, que como mencionamos es una medida de incerteza en una distribución de probabilidad.

**recordatorio**: no usar ciclos solo operaciones vectorizadas.

In [15]:
def calcular_entropia_total(p):
    entropy_additives = -1 * p * np.log2(p)
    return np.sum(entropy_additives)

x=([0.25,0.75])
calcular_entropia_total(x)



ValueError: operands could not be broadcast together with shapes (0,) (2,) 